In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

2023-07-22 16:03:41.888093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_data=pd.read_excel('Participants_Data/Data_Train.xlsx')
train_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [4]:
y=train_data['Price'].values

train_data=train_data.drop('Title',axis=1)
train_data=train_data.drop('Synopsis',axis=1)
train_data=train_data.drop('Edition',axis=1)
train_data=train_data.drop('Price',axis=1)

In [5]:
print('Few labels: ',y[:5])

Few labels:  [220.   202.93 299.   180.   965.62]


In [6]:
train_data.head()

,Author,Reviews,Ratings,Genre,BookCategory
0,Chris Kuzneski,4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure
1,Arun Khopkar,3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts"
2,Thomas Hobbes,4.8 out of 5 stars,6 customer reviews,International Relations,Humour
3,Agatha Christie,4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery"
4,Editors of Life,5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography"


Let's create buckets

In [10]:
import re

def preprocess(data):
    reviews=[]
    ratings=[]
    
    for each in ['Author','Genre','BookCategory']:
        data[each]=pd.Categorical(data[each])
        data[each]=data[each].cat.codes
    
    for each in data['Reviews'].values:
        reviews.append(float(each[:3]))
    
    for each in data['Ratings'].values:
        res=re.search(r"\d*",each)
        ratings.append(float(res[0]))
    
    reviews=pd.DataFrame(reviews)
    ratings=pd.DataFrame(ratings)
    data['Ratings']=ratings
    data['Reviews']=reviews
    return data

In [11]:
train=preprocess(train_data)
train.head()

,Author,Reviews,Ratings,Genre,BookCategory
0,615,4.0,8.0,1,0
1,307,3.9,14.0,74,2
2,3390,4.8,6.0,193,6
3,62,4.1,13.0,92,5
4,953,5.0,1.0,253,1


## Model training using sklearn Linear Regression module
Let's compare the result of a tf NN and sklearn Linear Regression module

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train= scaler.fit_transform(train)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Dense(8,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [14]:
model.fit(x_train, y, batch_size=128,epochs=2000)

Epoch 1/2000
49/49 [==============================] - 1s 977us/step - loss: 789910.6875
Epoch 2/2000
49/49 [==============================] - 0s 794us/step - loss: 789388.5000
Epoch 3/2000
49/49 [==============================] - 0s 685us/step - loss: 788469.0625
Epoch 4/2000
49/49 [==============================] - 0s 917us/step - loss: 786678.0000
Epoch 5/2000
49/49 [==============================] - 0s 1ms/step - loss: 783456.0625
Epoch 6/2000
49/49 [==============================] - 0s 859us/step - loss: 778498.3125
Epoch 7/2000
49/49 [==============================] - 0s 836us/step - loss: 771525.6250
Epoch 8/2000
49/49 [==============================] - 0s 754us/step - loss: 762382.4375
Epoch 9/2000
49/49 [==============================] - 0s 727us/step - loss: 750870.8750
Epoch 10/2000
49/49 [==============================] - 0s 689us/step - loss: 737001.3125
Epoch 11/2000
49/49 [==============================] - 0s 846us/step - loss: 720676.8750
Epoch 12/2000
49/49 [===========

49/49 [==============================] - 0s 671us/step - loss: 469985.5312
Epoch 94/2000
49/49 [==============================] - 0s 672us/step - loss: 469867.0938
Epoch 95/2000
49/49 [==============================] - 0s 665us/step - loss: 469783.4375
Epoch 96/2000
49/49 [==============================] - 0s 664us/step - loss: 469658.8125
Epoch 97/2000
49/49 [==============================] - 0s 663us/step - loss: 469570.5625
Epoch 98/2000
49/49 [==============================] - 0s 664us/step - loss: 469451.0625
Epoch 99/2000
49/49 [==============================] - 0s 661us/step - loss: 469325.6875
Epoch 100/2000
49/49 [==============================] - 0s 664us/step - loss: 469263.5938
Epoch 101/2000
49/49 [==============================] - 0s 674us/step - loss: 469128.0625
Epoch 102/2000
49/49 [==============================] - 0s 664us/step - loss: 469000.3750
Epoch 103/2000
49/49 [==============================] - 0s 681us/step - loss: 468923.5312
Epoch 104/2000
49/49 [=========

49/49 [==============================] - 0s 812us/step - loss: 464276.7500
Epoch 185/2000
49/49 [==============================] - 0s 859us/step - loss: 464217.0625
Epoch 186/2000
49/49 [==============================] - 0s 844us/step - loss: 464202.8125
Epoch 187/2000
49/49 [==============================] - 0s 844us/step - loss: 464172.3750
Epoch 188/2000
49/49 [==============================] - 0s 870us/step - loss: 464144.0625
Epoch 189/2000
49/49 [==============================] - 0s 947us/step - loss: 464129.0312
Epoch 190/2000
49/49 [==============================] - 0s 918us/step - loss: 464107.8750
Epoch 191/2000
49/49 [==============================] - 0s 869us/step - loss: 464086.3750
Epoch 192/2000
49/49 [==============================] - 0s 874us/step - loss: 464036.3438
Epoch 193/2000
49/49 [==============================] - 0s 887us/step - loss: 464009.5938
Epoch 194/2000
49/49 [==============================] - 0s 813us/step - loss: 463979.3438
Epoch 195/2000
49/49 [===

49/49 [==============================] - 0s 772us/step - loss: 461009.6250
Epoch 276/2000
49/49 [==============================] - 0s 791us/step - loss: 460994.3125
Epoch 277/2000
49/49 [==============================] - 0s 822us/step - loss: 460992.0938
Epoch 278/2000
49/49 [==============================] - 0s 781us/step - loss: 460903.8125
Epoch 279/2000
49/49 [==============================] - 0s 1ms/step - loss: 460875.1562
Epoch 280/2000
49/49 [==============================] - 0s 742us/step - loss: 460839.9688
Epoch 281/2000
49/49 [==============================] - 0s 717us/step - loss: 460823.7188
Epoch 282/2000
49/49 [==============================] - 0s 734us/step - loss: 460793.1875
Epoch 283/2000
49/49 [==============================] - 0s 713us/step - loss: 460820.2500
Epoch 284/2000
49/49 [==============================] - 0s 729us/step - loss: 460684.4062
Epoch 285/2000
49/49 [==============================] - 0s 768us/step - loss: 460727.7188
Epoch 286/2000
49/49 [=====

49/49 [==============================] - 0s 920us/step - loss: 458402.4688
Epoch 367/2000
49/49 [==============================] - 0s 988us/step - loss: 458361.3750
Epoch 368/2000
49/49 [==============================] - 0s 991us/step - loss: 458342.5312
Epoch 369/2000
49/49 [==============================] - 0s 968us/step - loss: 458327.1875
Epoch 370/2000
49/49 [==============================] - 0s 1ms/step - loss: 458295.1250
Epoch 371/2000
49/49 [==============================] - 0s 973us/step - loss: 458270.5312
Epoch 372/2000
49/49 [==============================] - 0s 949us/step - loss: 458285.5312
Epoch 373/2000
49/49 [==============================] - 0s 951us/step - loss: 458233.6250
Epoch 374/2000
49/49 [==============================] - 0s 990us/step - loss: 458241.5312
Epoch 375/2000
49/49 [==============================] - 0s 978us/step - loss: 458269.0938
Epoch 376/2000
49/49 [==============================] - 0s 933us/step - loss: 458205.3438
Epoch 377/2000
49/49 [=====

...

49/49 [==============================] - 0s 747us/step - loss: 445539.5000
Epoch 1634/2000
49/49 [==============================] - 0s 809us/step - loss: 445521.9688
Epoch 1635/2000
49/49 [==============================] - 0s 799us/step - loss: 445541.9062
Epoch 1636/2000
49/49 [==============================] - 0s 752us/step - loss: 445488.0938
Epoch 1637/2000
49/49 [==============================] - 0s 861us/step - loss: 445554.1562
Epoch 1638/2000
49/49 [==============================] - 0s 876us/step - loss: 445505.1875
Epoch 1639/2000
49/49 [==============================] - 0s 938us/step - loss: 445485.0938
Epoch 1640/2000
49/49 [==============================] - 0s 889us/step - loss: 445478.2500
Epoch 1641/2000
49/49 [==============================] - 0s 811us/step - loss: 445477.1562
Epoch 1642/2000
49/49 [==============================] - 0s 774us/step - loss: 445507.0938
Epoch 1643/2000
49/49 [==============================] - 0s 951us/step - loss: 445456.9062
Epoch 1644/2000

49/49 [==============================] - 0s 847us/step - loss: 445306.1562
Epoch 1724/2000
49/49 [==============================] - 0s 798us/step - loss: 445226.3438
Epoch 1725/2000
49/49 [==============================] - 0s 834us/step - loss: 445275.3750
Epoch 1726/2000
49/49 [==============================] - 0s 856us/step - loss: 445253.9062
Epoch 1727/2000
49/49 [==============================] - 0s 808us/step - loss: 445232.8750
Epoch 1728/2000
49/49 [==============================] - 0s 832us/step - loss: 445297.7500
Epoch 1729/2000
49/49 [==============================] - 0s 792us/step - loss: 445169.0938
Epoch 1730/2000
49/49 [==============================] - 0s 789us/step - loss: 445242.8438
Epoch 1731/2000
49/49 [==============================] - 0s 832us/step - loss: 445233.0625
Epoch 1732/2000
49/49 [==============================] - 0s 890us/step - loss: 445254.5312
Epoch 1733/2000
49/49 [==============================] - 0s 830us/step - loss: 445231.9062
Epoch 1734/2000

49/49 [==============================] - 0s 937us/step - loss: 445085.8750
Epoch 1814/2000
49/49 [==============================] - 0s 879us/step - loss: 445031.9375
Epoch 1815/2000
49/49 [==============================] - 0s 862us/step - loss: 445054.2188
Epoch 1816/2000
49/49 [==============================] - 0s 859us/step - loss: 445030.9688
Epoch 1817/2000
49/49 [==============================] - 0s 851us/step - loss: 445076.4688
Epoch 1818/2000
49/49 [==============================] - 0s 853us/step - loss: 445023.4688
Epoch 1819/2000
49/49 [==============================] - 0s 861us/step - loss: 445017.5625
Epoch 1820/2000
49/49 [==============================] - 0s 919us/step - loss: 445023.1562
Epoch 1821/2000
49/49 [==============================] - 0s 873us/step - loss: 445003.6875
Epoch 1822/2000
49/49 [==============================] - 0s 846us/step - loss: 445031.0938
Epoch 1823/2000
49/49 [==============================] - 0s 858us/step - loss: 445007.6250
Epoch 1824/2000

49/49 [==============================] - 0s 867us/step - loss: 444812.6250
Epoch 1904/2000
49/49 [==============================] - 0s 837us/step - loss: 444809.3750
Epoch 1905/2000
49/49 [==============================] - 0s 830us/step - loss: 444828.0625
Epoch 1906/2000
49/49 [==============================] - 0s 901us/step - loss: 444829.2188
Epoch 1907/2000
49/49 [==============================] - 0s 873us/step - loss: 444800.0312
Epoch 1908/2000
49/49 [==============================] - 0s 880us/step - loss: 444832.9062
Epoch 1909/2000
49/49 [==============================] - 0s 843us/step - loss: 444821.6562
Epoch 1910/2000
49/49 [==============================] - 0s 850us/step - loss: 444819.3125
Epoch 1911/2000
49/49 [==============================] - 0s 887us/step - loss: 444849.3125
Epoch 1912/2000
49/49 [==============================] - 0s 816us/step - loss: 444801.7500
Epoch 1913/2000
49/49 [==============================] - 0s 843us/step - loss: 444807.5625
Epoch 1914/2000

49/49 [==============================] - 0s 810us/step - loss: 444662.1875
Epoch 1994/2000
49/49 [==============================] - 0s 859us/step - loss: 444618.4062
Epoch 1995/2000
49/49 [==============================] - 0s 890us/step - loss: 444644.7500
Epoch 1996/2000
49/49 [==============================] - 0s 809us/step - loss: 444602.3438
Epoch 1997/2000
49/49 [==============================] - 0s 686us/step - loss: 444642.4062
Epoch 1998/2000
49/49 [==============================] - 0s 794us/step - loss: 444643.0938
Epoch 1999/2000
49/49 [==============================] - 0s 814us/step - loss: 444654.6250
Epoch 2000/2000
49/49 [==============================] - 0s 818us/step - loss: 444613.5938


In [15]:
#Submit part 
test_data=pd.read_excel('Participants_Data/Data_Test.xlsx')
test_data=test_data.drop('Title',axis=1)
test_data=test_data.drop('Synopsis',axis=1)
test_data=test_data.drop('Edition',axis=1)

In [16]:
submit=pd.read_excel('Participants_Data/Sample_Submission.xlsx')
submit.head()

,Price
0,119
1,123
2,108
3,118
4,132


In [17]:
test=preprocess(test_data)
x_test= scaler.fit_transform(test)

In [19]:
res=[]
for itr in range(len(x_test)):
  dat=x_test[itr]
  dat=np.expand_dims(dat,axis=0)
  res.append(model.predict(dat)[0][0])
  print(itr)

1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 18ms/step
2
1/1 [==============================] - 0s 19ms/step
3
1/1 [==============================] - 0s 19ms/step
4
1/1 [==============================] - 0s 19ms/step
5
1/1 [==============================] - 0s 19ms/step
6
1/1 [==============================] - 0s 17ms/step
7
1/1 [==============================] - 0s 17ms/step
8
1/1 [==============================] - 0s 17ms/step
9
1/1 [==============================] - 0s 17ms/step
10
1/1 [==============================] - 0s 17ms/step
11
1/1 [==============================] - 0s 17ms/step
12
1/1 [==============================] - 0s 17ms/step
13
1/1 [==============================] - 0s 18ms/step
14
1/1 [==============================] - 0s 19ms/step
15
1/1 [==============================] - 0s 18ms/step
16
1/1 [==============================] - 0s 19ms/step
17
1/1 [===============

1/1 [==============================] - 0s 17ms/step
148
1/1 [==============================] - 0s 17ms/step
149
1/1 [==============================] - 0s 18ms/step
150
1/1 [==============================] - 0s 17ms/step
151
1/1 [==============================] - 0s 17ms/step
152
1/1 [==============================] - 0s 17ms/step
153
1/1 [==============================] - 0s 18ms/step
154
1/1 [==============================] - 0s 18ms/step
155
1/1 [==============================] - 0s 19ms/step
156
1/1 [==============================] - 0s 18ms/step
157
1/1 [==============================] - 0s 18ms/step
158
1/1 [==============================] - 0s 18ms/step
159
1/1 [==============================] - 0s 16ms/step
160
1/1 [==============================] - 0s 17ms/step
161
1/1 [==============================] - 0s 16ms/step
162
1/1 [==============================] - 0s 18ms/step
163
1/1 [==============================] - 0s 18ms/step
164
1/1 [==============================] - 0s 17ms/s

1/1 [==============================] - 0s 17ms/step
294
1/1 [==============================] - 0s 17ms/step
295
1/1 [==============================] - 0s 16ms/step
296
1/1 [==============================] - 0s 16ms/step
297
1/1 [==============================] - 0s 17ms/step
298
1/1 [==============================] - 0s 20ms/step
299
1/1 [==============================] - 0s 17ms/step
300
1/1 [==============================] - 0s 17ms/step
301
1/1 [==============================] - 0s 18ms/step
302
1/1 [==============================] - 0s 20ms/step
303
1/1 [==============================] - 0s 17ms/step
304
1/1 [==============================] - 0s 18ms/step
305
1/1 [==============================] - 0s 17ms/step
306
1/1 [==============================] - 0s 17ms/step
307
1/1 [==============================] - 0s 17ms/step
308
1/1 [==============================] - 0s 18ms/step
309
1/1 [==============================] - 0s 17ms/step
310
1/1 [==============================] - 0s 19ms/s

1/1 [==============================] - 0s 19ms/step
440
1/1 [==============================] - 0s 18ms/step
441
1/1 [==============================] - 0s 18ms/step
442
1/1 [==============================] - 0s 18ms/step
443
1/1 [==============================] - 0s 20ms/step
444
1/1 [==============================] - 0s 18ms/step
445
1/1 [==============================] - 0s 17ms/step
446
1/1 [==============================] - 0s 17ms/step
447
1/1 [==============================] - 0s 16ms/step
448
1/1 [==============================] - 0s 17ms/step
449
1/1 [==============================] - 0s 18ms/step
450
1/1 [==============================] - 0s 18ms/step
451
1/1 [==============================] - 0s 17ms/step
452
1/1 [==============================] - 0s 17ms/step
453
1/1 [==============================] - 0s 17ms/step
454
1/1 [==============================] - 0s 17ms/step
455
1/1 [==============================] - 0s 18ms/step
456
1/1 [==============================] - 0s 17ms/s

1/1 [==============================] - 0s 17ms/step
586
1/1 [==============================] - 0s 16ms/step
587
1/1 [==============================] - 0s 17ms/step
588
1/1 [==============================] - 0s 18ms/step
589
1/1 [==============================] - 0s 18ms/step
590
1/1 [==============================] - 0s 17ms/step
591
1/1 [==============================] - 0s 18ms/step
592
1/1 [==============================] - 0s 17ms/step
593
1/1 [==============================] - 0s 17ms/step
594
1/1 [==============================] - 0s 16ms/step
595
1/1 [==============================] - 0s 15ms/step
596
1/1 [==============================] - 0s 15ms/step
597
1/1 [==============================] - 0s 15ms/step
598
1/1 [==============================] - 0s 16ms/step
599
1/1 [==============================] - 0s 15ms/step
600
1/1 [==============================] - 0s 16ms/step
601
1/1 [==============================] - 0s 15ms/step
602
1/1 [==============================] - 0s 16ms/s

...

1/1 [==============================] - 0s 24ms/step
878
1/1 [==============================] - 0s 19ms/step
879
1/1 [==============================] - 0s 18ms/step
880
1/1 [==============================] - 0s 19ms/step
881
1/1 [==============================] - 0s 17ms/step
882
1/1 [==============================] - 0s 17ms/step
883
1/1 [==============================] - 0s 17ms/step
884
1/1 [==============================] - 0s 16ms/step
885
1/1 [==============================] - 0s 17ms/step
886
1/1 [==============================] - 0s 16ms/step
887
1/1 [==============================] - 0s 17ms/step
888
1/1 [==============================] - 0s 18ms/step
889
1/1 [==============================] - 0s 17ms/step
890
1/1 [==============================] - 0s 18ms/step
891
1/1 [==============================] - 0s 18ms/step
892
1/1 [==============================] - 0s 18ms/step
893
1/1 [==============================] - 0s 17ms/step
894
1/1 [==============================] - 0s 18ms/s

1023
1/1 [==============================] - 0s 19ms/step
1024
1/1 [==============================] - 0s 20ms/step
1025
1/1 [==============================] - 0s 20ms/step
1026
1/1 [==============================] - 0s 18ms/step
1027
1/1 [==============================] - 0s 15ms/step
1028
1/1 [==============================] - 0s 29ms/step
1029
1/1 [==============================] - 0s 20ms/step
1030
1/1 [==============================] - 0s 19ms/step
1031
1/1 [==============================] - 0s 20ms/step
1032
1/1 [==============================] - 0s 19ms/step
1033
1/1 [==============================] - 0s 16ms/step
1034
1/1 [==============================] - 0s 15ms/step
1035
1/1 [==============================] - 0s 19ms/step
1036
1/1 [==============================] - 0s 18ms/step
1037
1/1 [==============================] - 0s 14ms/step
1038
1/1 [==============================] - 0s 14ms/step
1039
1/1 [==============================] - 0s 14ms/step
1040
1/1 [=====================

1/1 [==============================] - 0s 17ms/step
1167
1/1 [==============================] - 0s 17ms/step
1168
1/1 [==============================] - 0s 18ms/step
1169
1/1 [==============================] - 0s 18ms/step
1170
1/1 [==============================] - 0s 18ms/step
1171
1/1 [==============================] - 0s 17ms/step
1172
1/1 [==============================] - 0s 16ms/step
1173
1/1 [==============================] - 0s 16ms/step
1174
1/1 [==============================] - 0s 16ms/step
1175
1/1 [==============================] - 0s 17ms/step
1176
1/1 [==============================] - 0s 17ms/step
1177
1/1 [==============================] - 0s 17ms/step
1178
1/1 [==============================] - 0s 18ms/step
1179
1/1 [==============================] - 0s 16ms/step
1180
1/1 [==============================] - 0s 15ms/step
1181
1/1 [==============================] - 0s 16ms/step
1182
1/1 [==============================] - 0s 16ms/step
1183
1/1 [==========================

1/1 [==============================] - 0s 16ms/step
1310
1/1 [==============================] - 0s 16ms/step
1311
1/1 [==============================] - 0s 16ms/step
1312
1/1 [==============================] - 0s 16ms/step
1313
1/1 [==============================] - 0s 16ms/step
1314
1/1 [==============================] - 0s 16ms/step
1315
1/1 [==============================] - 0s 17ms/step
1316
1/1 [==============================] - 0s 18ms/step
1317
1/1 [==============================] - 0s 17ms/step
1318
1/1 [==============================] - 0s 17ms/step
1319
1/1 [==============================] - 0s 17ms/step
1320
1/1 [==============================] - 0s 16ms/step
1321
1/1 [==============================] - 0s 17ms/step
1322
1/1 [==============================] - 0s 17ms/step
1323
1/1 [==============================] - 0s 17ms/step
1324
1/1 [==============================] - 0s 18ms/step
1325
1/1 [==============================] - 0s 18ms/step
1326
1/1 [==========================

1/1 [==============================] - 0s 17ms/step
1453
1/1 [==============================] - 0s 20ms/step
1454
1/1 [==============================] - 0s 19ms/step
1455
1/1 [==============================] - 0s 17ms/step
1456
1/1 [==============================] - 0s 20ms/step
1457
1/1 [==============================] - 0s 19ms/step
1458
1/1 [==============================] - 0s 19ms/step
1459
1/1 [==============================] - 0s 18ms/step
1460
1/1 [==============================] - 0s 17ms/step
1461
1/1 [==============================] - 0s 17ms/step
1462
1/1 [==============================] - 0s 18ms/step
1463
1/1 [==============================] - 0s 17ms/step
1464
1/1 [==============================] - 0s 17ms/step
1465
1/1 [==============================] - 0s 18ms/step
1466
1/1 [==============================] - 0s 16ms/step
1467
1/1 [==============================] - 0s 18ms/step
1468
1/1 [==============================] - 0s 17ms/step
1469
1/1 [==========================

In [20]:
res=pd.DataFrame(res)
submit['Price']=res
submit.to_csv('Participants_Data/solution.csv',index=False)